## Run runoff projections ...

creates the runoff projection file:`./01_data/4_runoff_volume_proj_common_running_refs_w5e5_isimip3b.nc`

- basically similar to `./00_data_creating_scripts/04_run_volume_projections.ipynb`
    - but I did not use run_with_hydro there, so I needed to repeat it here 
    - (if I would only run run_with_hydro, I would not need to actually run the run_volume_projections script. However, the here created runoff and volume projections only go until 2099 as runoff needs different data, so I still use the other script for volume projections) 

In [1]:
# path where OGGM gdirs are: is created in data_creating_scripts
working_dir = '/home/users/lschuster/Schuster_et_al_phd_paper_1_cluster/oggm_run_gdir_folder/node_folder'
# figure path
fig_path = '/home/users/lschuster/Schuster_et_al_phd_paper_1_cluster/figures'
# data path -> data created from 00_data_creating scripts
data_path = '/home/users/lschuster/Schuster_et_al_phd_paper_1_cluster/01_data'

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr

plt.rc('font', size=20)
import warnings
warnings.filterwarnings("once", category=DeprecationWarning)  # noqa: E402
import scipy
# imports from OGGM
import oggm
from oggm import utils, workflow, tasks, cfg, entity_task
import numpy as np
import pandas as pd
from MBsandbox.mbmod_daily_oneflowline import (TIModel, TIModel_Sfc_Type, process_w5e5_data,
                                              compile_fixed_geometry_mass_balance_TIModel,
                                              extend_past_climate_run_TIModel)
from MBsandbox.help_func import (minimize_winter_mb_brentq_geod_via_pf, minimize_bias_geodetic,
                                 calibrate_to_geodetic_bias_winter_mb)
from MBsandbox.flowline_TIModel import run_from_climate_data_TIModel
import time
import logging

log = logging.getLogger(__name__)    
    
_, pathi = utils.get_wgms_files()
pd_mb_overview = pd.read_csv(pathi[:-len('/mbdata')] + '/mb_overview_seasonal_mb_time_periods_20220301.csv',
                             index_col='Unnamed: 0')
pd_wgms_data_stats = pd.read_csv(pathi[:-len('/mbdata')] + '/wgms_data_stats_20220301.csv', index_col='Unnamed: 0')

# should have at least 5 annual MB estimates in the time period 1980-2019
# (otherwise can also not have MB profiles or winter MB!)
pd_wgms_data_stats = pd_wgms_data_stats.loc[pd_wgms_data_stats.len_annual_balance>=5]
ref_candidates = pd_wgms_data_stats.rgi_id.unique()

In [3]:
# compute runoff only for the 88 glaciers that run for all calib and MB model options ...
pd_params_stats_working_for_all = pd.read_csv(f'{data_path}/0_pd_params_stats_working_for_all.csv')

ref_candidates = pd_params_stats_working_for_all.rgi_id.unique()

In [5]:
cfg.initialize()
cfg.PARAMS['use_multiprocessing'] = True #True
cfg.PATHS['working_dir'] = working_dir
cfg.PARAMS['hydro_month_nh'] = 1
cfg.PARAMS['hydro_month_sh'] = 1
cfg.PARAMS['continue_on_error'] = True
# then repeat it for pseudo_daily and mb_pseudo_daily_fake for all things except False ...
cfg.PARAMS['mp_processes'] = 28

warnings.filterwarnings("ignore", category=DeprecationWarning) 
gdirs = workflow.init_glacier_directories(ref_candidates)

2023-01-13 15:44:08: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-01-13 15:44:08: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-01-13 15:44:08: oggm.cfg: Multiprocessing: using all available processors (N=32)
2023-01-13 15:44:08: oggm.cfg: Multiprocessing switched ON after user settings.
2023-01-13 15:44:08: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2023-01-13 15:44:08: oggm.cfg: Multiprocessing: using the requested number of processors (N=28)
2023-01-13 15:44:08: oggm.workflow: Execute entity tasks [GlacierDirectory] on 88 glaciers


In [6]:
_doc = 'the calibrated melt_f, pf and temp.b to match geodetic mean observation, winter MB and approximately std of direct annual time series'
cfg.BASENAMES['calib_geod_opt_winter_mb_approx_std'] = ('calib_geod_opt_winter_mb_approx_std.json', _doc)

_doc2 = 'the calibrated melt_f and pf to match geodetic mean observation and winter MB bias, temp. bias set to zero!'
cfg.BASENAMES['calib_geod_opt_winter_mb_temp_b_0'] = ('calib_geod_opt_winter_mb_temp_b_0.json', _doc2)

_doc3 = 'the calibrated melt_f and pf to match geodetic mean observation and interannual direct glaciological standard deviation, temp. bias set to zero!'
cfg.BASENAMES['calib_geod_opt_std_temp_b_0'] = ('calib_geod_opt_std_temp_b_0.json', _doc3)

_doc4 = 'the calibrated melt_f to match geodetic mean observation, no temp. bias and globally cte prcp. fac extracted from median of ref glacier matching direct glaciological standard deviation'
cfg.BASENAMES['calib_only_geod_temp_b_0_pf_cte_via_std'] = ('calib_only_geod_temp_b_0_pf_cte_via_std.json', _doc4)

_doc5 = 'the calibrated melt_f to match geodetic mean observation, no temp. bias and per glacier one prcp. factor extracted from median of ref glacier matching direct glaciological standard deviation'
cfg.BASENAMES['calib_only_geod_temp_b_0_pf_fit_via_winter_mb'] = ('calib_only_geod_temp_b_0_pf_fit_via_winter_mb.json', _doc5)

In [7]:
calib_types = ['calib_geod_opt_winter_mb_approx_std',
               'calib_geod_opt_winter_mb_temp_b_0',
               'calib_geod_opt_std_temp_b_0',
               'calib_only_geod_temp_b_0_pf_cte_via_std',
               'calib_only_geod_temp_b_0_pf_fit_via_winter_mb']

In [8]:
cfg.PARAMS['continue_on_error'] = True
oggm.cfg.set_logging_config('CRITICAL')
gcms = ['ukesm1-0-ll','gfdl-esm4', 'ipsl-cm6a-lr', 'mpi-esm1-2-hr', 'mri-esm2-0' ]
climate_type = 'W5E5'
cfg.PARAMS['store_fl_diagnostics'] = True


**Projection run with runoff**: (equal to the one without but, use `run_with_hydro` instead)
- get the calibration
- do the inversion and find a glen_a that fits Farinotti(2019) for every glacier
- run historical volume & runoff components projections with W5E5
- and future volume & runoff components projections with ISIMIP3b for the two SSPs and 5 GCMs

In [9]:
cfg.PARAMS['mp_processes'] = 29
melt_f_update = 'monthly'
load = False
if load:
    for mb_type in ['mb_monthly', 'mb_pseudo_daily',
                    'mb_pseudo_daily_fake','mb_real_daily']: #['mb_monthly']: #, 'mb_pseudo_daily', 'mb_pseudo_daily_fake','mb_real_daily']: #['mb_pseudo_daily_fake']: #
        if mb_type == 'mb_real_daily':
            temp_resol = 'daily'
        else:
            temp_resol = 'monthly'

        for grad_type in ['cte', 'var_an_cycle']:
            for calib_type in calib_types:
                for melt_f_change in [False, 'neg_exp', 'linear']: #[False]: #, 'neg_exp', 'linear']: [False]: #, 'neg_exp', 'linear']:
                    if melt_f_change is False:
                        mb_model_sub_class = TIModel #TIModel_Sfc_Type
                        kwargs_for_TIModel_Sfc_Type = {}
                    else:
                        mb_model_sub_class = TIModel_Sfc_Type
                        kwargs_for_TIModel_Sfc_Type={'melt_f_change': melt_f_change,
                                                     'melt_f_update': melt_f_update,
                                                     'tau_e_fold_yr':1,
                                                     }
                        
                    for gdir in gdirs: #gdirs_sel:
                        try:
                            json_filename = calib_type
                            from_json = True
                            # get the apparent_mb (necessary for inversion)
                            ye = 2020
                            residual = 0
                            if from_json:
                                if melt_f_change is not False:
                                    if kwargs_for_TIModel_Sfc_Type['melt_f_update'] == 'annual':
                                        fs_new = '_{}_sfc_type_{}_annual_{}_{}'.format('W5E5', melt_f_change, mb_type, grad_type)
                                    else:
                                        fs_new = '_{}_sfc_type_{}_{}_{}'.format('W5E5', melt_f_change, mb_type, grad_type)
                                else:
                                    fs_new = '_{}_sfc_type_{}_{}_{}'.format('W5E5', melt_f_change, mb_type, grad_type)
                                # json_filename = 'melt_f_geod_opt_winter_mb_approx_std'
                                # get the calibrated melt_f that suits to the prcp factor
                                try:
                                    d = gdir.read_json(filename=json_filename,
                                                       filesuffix=fs_new)
                                    # get the corrected ref_hgt so that we can apply this again on the mb model
                                    # if otherwise not melt_f could be found!
                                    pf = d['pf']
                                    melt_f = d['melt_f']
                                    temp_bias = d['temp_bias']
                                except:
                                    raise InvalidWorkflowError(
                                        'there is no calibrated melt_f for this precipitation factor, glacier, climate'
                                        'mb_type and grad_type, need to do the calibration first!')


                            mb = mb_model_sub_class(gdir, None, mb_type=mb_type,
                                                    grad_type=grad_type,
                                                    baseline_climate=climate_type, residual=residual,
                                                    **kwargs_for_TIModel_Sfc_Type)
                            mb.melt_f = melt_f
                            mb.prcp_fac = pf
                            mb.temp_bias = temp_bias

                            oggm.core.climate.apparent_mb_from_any_mb(gdir, mb_model=mb,
                                                            mb_years=np.arange(2000, ye, 1))
                            # here glen-a is calibrated to match gdirs glaciers in total (as there is only one glacier, it just matches perfectly the one glacier!)
                            border = 80
                            filter = border >= 20
                            pd_inv_melt_f = oggm.workflow.calibrate_inversion_from_consensus([gdir],
                                                                                  apply_fs_on_mismatch=False,ignore_missing=False,
                                                                                  error_on_mismatch=True,
                                                                                  filter_inversion_output=filter)
                        except:
                            pass

                    # so for init_present_time_glacier, automatically the new glen a volume inversion is used!
                    workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs) #gdirs_sel)
                    y0 = gdirs[0].get_climate_info('_daily_W5E5')['baseline_hydro_yr_0']
                    # One adds 1 because the run ends at the end of the year
                    ye = gdirs[0].get_climate_info('_daily_W5E5')['baseline_hydro_yr_1'] + 1
                    
                    
                    output_filesuffix = f'hydro_historical_W5E5_{calib_type}_{mb_type}_{grad_type}_{melt_f_change}_{melt_f_update}'
                    workflow.execute_entity_task(run_with_hydro,
                                                 gdirs, run_task=run_from_climate_data_TIModel,
                                                     bias=0,  # will actually point to the residual, should always be zero! 
                                                     mb_model_sub_class=mb_model_sub_class,  # we use the temperature-index model variant with surface type distinction!
                                                     min_ys=y0,
                                                     ye=ye,  # starting and end year of the volume run 
                                                     mb_type=mb_type,
                                                     grad_type=grad_type,
                                                     precipitation_factor=None,  # take the fitted precipitation factor
                                                     melt_f=None, # set to the calibrated melt_f (that fits to fit_prcp_fac_aneto)
                                                     kwargs_for_TIModel_Sfc_Type=kwargs_for_TIModel_Sfc_Type,                             
                                                     climate_filename='climate_historical',
                                                     output_filesuffix=output_filesuffix,
                                                     climate_input_filesuffix='W5E5', 
                                                     store_model_geometry = True,
                                                     all_from_json=True,
                                                     json_filename=calib_type,
                                                 no_qc=True)
                    for ensemble in gcms:
                        if ensemble == 'ukesm1-0-ll':
                            ensemble = ensemble + '_r1i1p1f2'
                        else:
                            ensemble = ensemble + '_r1i1p1f1'
                        for ssp in ['ssp585', 'ssp126']:

                            workflow.execute_entity_task(run_with_hydro,
                             gdirs, run_task=run_from_climate_data_TIModel,  # gdirs_sel
                                                         init_model_filesuffix=output_filesuffix,
                                                         ref_geometry_filesuffix=output_filesuffix,  # also use this as area reference!!!!
                                                         ref_area_from_y0=True, # keep the same reference area as historical time period!!!
                                                         bias=0,  # will actually point to the residual, should always be zero! 
                                                         mb_model_sub_class=mb_model_sub_class,  # we use the temperature-index model variant with surface type distinction!
                                                         ys=2019,
                                                         ye=2100,  # starting and end year of the volume run 
                                                         mb_type=mb_type,
                                                         grad_type=grad_type,
                                                         precipitation_factor=None,  # take the fitted precipitation factor
                                                         melt_f=None, # set to the calibrated melt_f (that fits to fit_prcp_fac_aneto)
                                                         #init_model_filesuffix = '_historical_run',  # start from the stage of the end of the historical run 
                                                         #climate_type = 'gcm_data',
                                                         kwargs_for_TIModel_Sfc_Type=kwargs_for_TIModel_Sfc_Type,                             
                                                         climate_filename='gcm_data',
                                                         climate_input_filesuffix=f'ISIMIP3b_{ensemble}_{ssp}_no_correction', 
                                                         output_filesuffix=f'hydro_gcm_run_{ensemble}_{ssp}_{calib_type}_{mb_type}_{grad_type}_{melt_f_change}_{melt_f_update}', # can add here more options to distinguish between runs
                                                         store_model_geometry = True,
                                                         all_from_json=True,
                                                         json_filename=calib_type,
                                                         no_qc=True)
        print(mb_type)

-> need to concatenate them similar as in that here below:

In [17]:
runoff_vars_vol = ['volume','melt_off_glacier', 'melt_on_glacier',
                   'liq_prcp_off_glacier', 'liq_prcp_on_glacier']
runoff_vars = ['melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier']
load = False
if load:
    utils.compile_glacier_statistics(gdirs)
    ds_merged_l_gcms_hydro = []

    ds_dict = {}
    working_rgis = []
    ds_merged_l = []
    ds_merged_nn_l = []
    for grad_type in ['cte', 'var_an_cycle']:
        ds_merged_n_l = []
        for mb_type in ['mb_monthly', 'mb_pseudo_daily', 'mb_pseudo_daily_fake', 'mb_real_daily']:
            ds_merged_l = []
            for ssp in ['ssp126', 'ssp585']:
                ds_merged_list = []
                for j,calib_type in enumerate(calib_types):
                    ds_list = [] # commented as I don't have melt_f_change
                    
                    for melt_f_change in [False, 'neg_exp','linear']: # commented as I don't have melt_f_change
                        ds_runoff_hist = utils.compile_run_output(gdirs, 
                                         input_filesuffix=f'hydro_historical_W5E5_{calib_type}_{mb_type}_{grad_type}_{melt_f_change}_{melt_f_update}')
                        
                        ds_runoff_hist = ds_runoff_hist.isel(time=slice(0, -1)).load()[runoff_vars_vol]
                        
                        ds_list0 = []
                        for ensemble in gcms:
                            if ensemble == 'ukesm1-0-ll':
                                ensemble = ensemble + '_r1i1p1f2'
                            else:
                                ensemble = ensemble + '_r1i1p1f1'
                            path_gcm_hydro = f'hydro_gcm_run_{ensemble}_{ssp}_{calib_type}_{mb_type}_{grad_type}_{melt_f_change}_{melt_f_update}'
                            ds_runoff_gcm = utils.compile_run_output(gdirs,
                                                                     input_filesuffix=path_gcm_hydro)
                            ds_runoff_gcm = ds_runoff_gcm.isel(time=slice(0, -1)).load()[runoff_vars_vol]
                            ds = xr.concat([ds_runoff_hist,ds_runoff_gcm],dim='time').dropna(dim='rgi_id', how='all')
                            ds['gcm'] = ensemble
                            #ds['ssp'] = ssp
                            working_rgis.append(ds.rgi_id.values)
                            ds_dict[f'{calib_type}_{melt_f_change}'] = ds
                            ds_list0.append(ds)
                            #import sys
                            #sys.exit()
                        ds_list_s = xr.concat(ds_list0, dim='gcm')
                        ds_list_s['melt_f_change'] = str(melt_f_change) #.astype(str)
                        ds_list.append(ds_list_s) # commented as I don't have melt_f_change
                    ds_t = xr.concat(ds_list, dim='melt_f_change') # commented as I don't have melt_f_change
                    ds_t['calib_type'] = calib_type  # commented as I don't have melt_f_change
                    ds_merged_list.append(ds_t)

                    #ds_list_s['calib_type'] = calib_type
                    #    ds_merged_list.append(ds_list_s)
                ds_merged_t = xr.concat(ds_merged_list, dim='calib_type')
                ds_merged_t['ssp'] = ssp
                ds_merged_l.append(ds_merged_t)
            ds_merged_n = xr.concat(ds_merged_l, dim = 'ssp')
            ds_merged_n['mb_type'] = mb_type
            ds_merged_n_l.append(ds_merged_n)
        ds_merged_nn = xr.concat(ds_merged_n_l, dim='mb_type')
        ds_merged_nn['grad_type'] = grad_type
        ds_merged_nn_l.append(ds_merged_nn)
    ds_merged = xr.concat(ds_merged_nn_l, dim='grad_type')

    working_rgis_common = working_rgis[0]
    for j,k in enumerate(working_rgis[1:]):
        working_rgis_common = list(set(working_rgis_common).intersection(working_rgis[j]))
    ds_merged_gcms = ds_merged.sel(rgi_id = working_rgis_common)
    # vol_pathi = f'./trial_or_old_notebooks/all_refs_volume_projections.nc'
    #ds_merged.to_netcdf(path=vol_pathi)
    
    # We only want to save those glacier runoff projections that work for all options:
    
    # ds_merged_gcms = ds_merged_gcms.astype(np.float32)
    ds_merged_gcms = ds_merged_gcms.sel(time=slice(2000,2101))
    ds_merged_gcms = ds_merged_gcms.drop(['hydro_year','hydro_month','calendar_year', 'calendar_month'])
    ds_merged_gcms.coords['time'] = ds_merged_gcms.time.astype(int)
    ds_merged_gcms.attrs['used climate data'] = 'W5E5 until 2019 + ISIMIP3b (2020-2099)'
    ds_merged_gcms.attrs['created by'] = 'Schuster et al. (2023) with OGGM using the OGGM massbalance-sandbox'
    
    ds_merged_gcms.to_netcdf(f'{data_path}/4_runoff_volume_proj_common_running_refs_w5e5_isimip3b.nc')
    

In [13]:
#ds_merged_gcms = xr.open_dataset(f'{data_path}/projections_all_calib_timodel_common_running_rgis_runoff_volume_w_pseudo_daily_fake.nc')

In [4]:
ds_merged_gcms = xr.open_dataset(f'{data_path}/4_runoff_volume_proj_common_running_refs_w5e5_isimip3b.nc')

/home/users/lschuster/.local/lib/python3.10/site-packages/xarray/backends/plugins.py:105: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entrypoints = entry_points().get("xarray.backends", ())


In [5]:
ds_merged_gcms

<xarray.Dataset>
Dimensions:               (time: 101, rgi_id: 85, gcm: 5, melt_f_change: 3,
                           grad_type: 2, mb_type: 4, ssp: 2, calib_type: 5)
Coordinates:
  * time                  (time) int64 2000 2001 2002 2003 ... 2097 2098 2099
  * rgi_id                (rgi_id) object 'RGI60-01.09162' ... 'RGI60-06.00238'
  * gcm                   (gcm) object 'ukesm1-0-ll_r1i1p1f2' ... 'mri-esm2-0...
  * melt_f_change         (melt_f_change) object 'False' 'neg_exp' 'linear'
  * calib_type            (calib_type) object 'calib_geod_opt_winter_mb_appro...
  * ssp                   (ssp) object 'ssp126' 'ssp585'
  * mb_type               (mb_type) object 'mb_monthly' ... 'mb_real_daily'
  * grad_type             (grad_type) object 'cte' 'var_an_cycle'
Data variables:
    volume                (grad_type, mb_type, ssp, calib_type, melt_f_change, gcm, time, rgi_id) float64 ...
    melt_off_glacier      (grad_type, mb_type, ssp, calib_type, melt_f_change, gcm, time, rgi_id) float64 ...
    melt_on_glacier       (grad_type, mb_type, ssp, calib_type, melt_f_change, gcm, time, rgi_id) float64 ...
    liq_prcp_off_glacier  (grad_type, mb_type, ssp, calib_type, melt_f_change, gcm, time, rgi_id) float64 ...
    liq_prcp_on_glacier   (grad_type, mb_type, ssp, calib_type, melt_f_change, gcm, time, rgi_id) float64 ...
Attributes:
    description:        OGGM model output
    oggm_version:       1.5.4.dev52+ga3123b5
    calendar:           365-day no leap
    creation_date:      2023-01-02 09:57:05
    used climate data:  W5E5 until 2019 + ISIMIP3b (2020-2099)
    created by:         Schuster et al. (2023) with OGGM using the OGGM massb...